# T1574.006 Dynamic Linker Hijacking

-----------------------------------------------------------------------

## Technique Description

Adversaries may execute their own malicious payloads by hijacking environment variables the dynamic linker uses to load shared libraries. During the execution preparation phase of a program, the dynamic linker loads specified absolute paths of shared libraries from environment variables and files, such as <code>LD_PRELOAD</code> on Linux or <code>DYLD_INSERT_LIBRARIES</code> on macOS. Libraries specified in environment variables are loaded first, taking precedence over system libraries with the same function name.(Citation: Man LD.SO)(Citation: TLDP Shared Libraries)(Citation: Apple Doco Archive Dynamic Libraries) These variables are often used by developers to debug binaries without needing to recompile, deconflict mapped symbols, and implement custom functions without changing the original library.(Citation: Baeldung LD_PRELOAD)

On Linux and macOS, hijacking dynamic linker variables may grant access to the victim process's memory, system/network resources, and possibly elevated privileges. This method may also evade detection from security products since the execution is masked under a legitimate process. Adversaries can set environment variables via the command line using the <code>export</code> command, <code>setenv</code> function, or <code>putenv</code> function. Adversaries can also leverage [Dynamic Linker Hijacking](https://attack.mitre.org/techniques/T1574/006) to export variables in a shell or set variables programmatically using higher level syntax such Python’s <code>os.environ</code>.

On Linux, adversaries may set <code>LD_PRELOAD</code> to point to malicious libraries that match the name of legitimate libraries which are requested by a victim program, causing the operating system to load the adversary's malicious code upon execution of the victim program. <code>LD_PRELOAD</code> can be set via the environment variable or <code>/etc/ld.so.preload</code> file.(Citation: Man LD.SO)(Citation: TLDP Shared Libraries) Libraries specified by <code>LD_PRELOAD</code> are loaded and mapped into memory by <code>dlopen()</code> and <code>mmap()</code> respectively.(Citation: Code Injection on Linux and macOS)(Citation: Uninformed Needle) (Citation: Phrack halfdead 1997)(Citation: Brown Exploiting Linkers) 

On macOS this behavior is conceptually the same as on Linux, differing only in how the macOS dynamic libraries (dyld) is implemented at a lower level. Adversaries can set the <code>DYLD_INSERT_LIBRARIES</code> environment variable to point to malicious libraries containing names of legitimate libraries or functions requested by a victim program.(Citation: TheEvilBit DYLD_INSERT_LIBRARIES)(Citation: Timac DYLD_INSERT_LIBRARIES)(Citation: Gabilondo DYLD_INSERT_LIBRARIES Catalina Bypass) 

## Technique Detection

Monitor for changes to environment variables and files associated with loading shared libraries such as <code>LD_PRELOAD</code> and <code>DYLD_INSERT_LIBRARIES</code>, as well as the commands to implement these changes.

Monitor processes for unusual activity (e.g., a process that does not use the network begins to do so). Track library metadata, such as a hash, and compare libraries that are loaded at process execution time against previous executions to detect differences that do not correlate with patching or updates.

-----------------------------------------------------------------------

### Tactics:

  * Persistence

  * Privilege-Escalation

  * Defense-Evasion

### Platforms:

  * Linux

  * macOS

### Adversary Required Permissions:

  * User

### Data Sources:

  * **Process:** Process Creation

  * **File:** File Modification

  * **Module:** Module Load

  * **File:** File Creation

  * **Command:** Command Execution

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| Rocke | [Rocke](https://attack.mitre.org/groups/G0106) has modified /etc/ld.so.preload to hook libc functions in order to hide the installed dropper and mining software in process lists.(Citation: Anomali Rocke March 2019)	| 
| APT41 | [APT41](https://attack.mitre.org/groups/G0096) has configured payloads to load via LD_PRELOAD.(Citation: Crowdstrike GTR2020 Mar 2020)	| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1574/006)

  * [Capec](https://capec.mitre.org/data/definitions/13.html)

  * [Capec](https://capec.mitre.org/data/definitions/640.html)

  * [Man Ld.So](https://www.man7.org/linux/man-pages/man8/ld.so.8.html), Kerrisk, M. (2020, June 13). Linux Programmer's Manual. Retrieved June 15, 2020.

  * [Tldp Shared Libraries](https://www.tldp.org/HOWTO/Program-Library-HOWTO/shared-libraries.html), The Linux Documentation Project. (n.d.). Shared Libraries. Retrieved January 31, 2020.

  * [Apple Doco Archive Dynamic Libraries](https://developer.apple.com/library/archive/documentation/DeveloperTools/Conceptual/DynamicLibraries/100-Articles/OverviewOfDynamicLibraries.html), Apple Inc.. (2012, July 23). Overview of Dynamic Libraries. Retrieved March 24, 2021.

  * [Baeldung Ld_Preload](https://www.baeldung.com/linux/ld_preload-trick-what-is), baeldung. (2020, August 9). What Is the LD_PRELOAD Trick?. Retrieved March 24, 2021.

  * [Code Injection On Linux And Macos](https://www.datawire.io/code-injection-on-linux-and-macos/), Itamar Turner-Trauring. (2017, April 18). “This will only hurt for a moment”: code injection on Linux and macOS with LD_PRELOAD. Retrieved December 20, 2017.

  * [Uninformed Needle](http://hick.org/code/skape/papers/needle.txt), skape. (2003, January 19). Linux x86 run-time process manipulation. Retrieved December 20, 2017.

  * [Phrack Halfdead 1997](http://phrack.org/issues/51/8.html), halflife. (1997, September 1). Shared Library Redirection Techniques. Retrieved December 20, 2017.

  * [Brown Exploiting Linkers](http://www.nth-dimension.org.uk/pub/BTL.pdf), Tim Brown. (2011, June 29). Breaking the links: Exploiting the linker. Retrieved March 29, 2021.

  * [Theevilbit Dyld_Insert_Libraries](https://theevilbit.github.io/posts/dyld_insert_libraries_dylib_injection_in_macos_osx_deep_dive/), Fitzl, C. (2019, July 9). DYLD_INSERT_LIBRARIES DYLIB injection in macOS / OSX. Retrieved March 26, 2020.

  * [Timac Dyld_Insert_Libraries](https://blog.timac.org/2012/1218-simple-code-injection-using-dyld_insert_libraries/), Timac. (2012, December 18). Simple code injection using DYLD_INSERT_LIBRARIES. Retrieved March 26, 2020.

  * [Gabilondo Dyld_Insert_Libraries Catalina Bypass](https://jon-gabilondo-angulo-7635.medium.com/how-to-inject-code-into-mach-o-apps-part-ii-ddb13ebc8191), Jon Gabilondo. (2019, September 22). How to Inject Code into Mach-O Apps. Part II.. Retrieved March 24, 2021.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------
### This technique is a duplicate.  Follow the link below to the "Primary Version".
<a href="../Persistence/T1574.006 Dynamic Linker Hijacking.ipynb" target="_blank">Primary Version</a>